In [1]:
import os
import matplotlib.pyplot as plt
import jax
import jax.numpy as np
import params
import model
import convert
import stimulus

In [2]:
def show_figure(states, times, figsize, dt, rows=5):
    cols = (len(states) % rows) + 1
    fig, ax = plt.subplots(cols, rows, figsize=figsize)
    idx = 0
    for col in range(cols):
        for row in range(rows):
            if idx >= len(states):
                return
            ax[col, row].imshow(states[idx], cmap="magma", vmin=0, vmax=1,)
            ax[col, row].set_title("Iter: " + str(times[idx + 1]) + " (%sms)" % convert.units_to_ms(times[idx + 1], dt))
            idx += 1

In [3]:
# simulation inputs (real values)
field_size = (12.5, 12.5)  # cm
d = 0.001  # (cm^2/ms)
cell_parameters = params.params3()

In [ ]:
# infinitesimals
dx = 0.015  # (cm/units) - Fenton 1998 recommends ~200, 300 micron/gridunit (~0.02, 0.03), smaller dx means finer grid
dt = 0.02  # (ms) - Fenton 1998 recommends few hundreds of ms (~0.01, 0.04)

In [ ]:
# to computational units
shape = convert.field_to_shape(field_size, dx)

stripe_size = int(shape[0] / 10)

protocol1 = stimulus.protocol(start=0, duration=2, period=0)
s1 = stimulus.rectangular(shape, jax.ops.index[:, -stripe_size:], 1., protocol1)

protocol2 = stimulus.protocol(start=convert.ms_to_units(400, dt), duration=2, period=0)
s2 = stimulus.rectangular(shape, jax.ops.index[-400:], 1., protocol2)

stimuli = [s1, s2]

checkpoints = [0, 350, 385, 405, 425, 445, 460, 475, 495, 510, 525, 575, 625, 910, 1115, 1160, 1210, 1260, 1300, 1340, 1390, 1420, 1480, 1530, 1600]
checkpoints = [convert.ms_to_units(ck, dt) for ck in checkpoints]

print(shape)
print("real tissue size:", field_size)
print("Checkpointing at:", checkpoints)
print(cell_parameters)
model.show_stimuli(*stimuli)

In [ ]:
state = model.init(shape)
states = []
for i in range(len(checkpoints) - 1):
    state = model._forward(state, checkpoints[i], checkpoints[i + 1], cell_parameters, np.ones(shape) * d, stimuli, dt, dx)
    u = state[2]
    states.append(u)
#     model.show(state)
          
show_figure(states, checkpoints, (20, 20), dt)